In [1]:
import pandas as pd
import re

In [2]:
df1=pd.read_csv(r"Tweets\tweets_14-08_to_23-08.csv", parse_dates=["fecha"])
df2=pd.read_csv(r"Tweets\tweets_21-08_to_30-08.csv", parse_dates=["fecha"])
df3=pd.read_csv(r"Tweets\tweets_28-08_to_05-09.csv", parse_dates=["fecha"])
df4=pd.read_csv(r"Tweets\tweets_04-09_to_13-09.csv", parse_dates=["fecha"])

In [3]:
df=pd.concat([df1,df2,df3,df4],axis=0)

In [4]:
df=df.drop_duplicates("ID").reset_index(drop=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34559 entries, 0 to 34558
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           34559 non-null  int64         
 1   tweet        34559 non-null  object        
 2   fecha        34559 non-null  datetime64[ns]
 3   usuario      34559 non-null  object        
 4   dispositivo  34557 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 945.0+ KB


##### Cambiar el formato de la fecha y convertir el tiempo de UTC a la zona horaria del centro (UTC -5) y crear columnas para mes, día y hora

In [6]:
df["fecha"]=df["fecha"].dt.tz_localize('utc').dt.tz_convert('US/Central')
df["fecha_str"]=df["fecha"].apply(lambda x: x.strftime("%d-%m"))
df["mes"]=df["fecha"].apply(lambda x: x.month)
df["dia"]=df["fecha"].apply(lambda x: x.day)
df["hora"]=df["fecha"].apply(lambda x: x.hour)

##### Eliminar los hashtags y crear una columna con ellos

In [7]:
hashtags=re.compile(r"#[\w]+")
df["hashtags"]=df["tweet"].apply(lambda x: hashtags.findall(x))
df["tweet"]=df["tweet"].apply(lambda x: hashtags.sub("",x))

##### Eliminar los user tags y crear una columna con ellos

In [8]:
users=re.compile(r"@[\w]+")
df["users"]=df["tweet"].apply(lambda x: users.findall(x))
df["tweet"]=df["tweet"].apply(lambda x: users.sub("",x))

##### Eliminar los links 

In [9]:
links=re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")
df["tweet"]=df["tweet"].apply(lambda x: links.sub("",x))

##### Convertir el texto en minúsculas, eliminar los acentos y separadores en formato unicode 

In [10]:
df["tweet"]=df["tweet"].apply(lambda x: x.lower())
df["tweet"]=df["tweet"].apply(lambda x: x.replace("á","a"))
df["tweet"]=df["tweet"].apply(lambda x: x.replace("é","e"))
df["tweet"]=df["tweet"].apply(lambda x: x.replace("í","i"))
df["tweet"]=df["tweet"].apply(lambda x: x.replace("ó","o"))
df["tweet"]=df["tweet"].apply(lambda x: x.replace("ú","u"))
df["tweet"]=df["tweet"].apply(lambda x: x.replace(u'\xa0', u''))
df["tweet"]=df["tweet"].apply(lambda x: x.replace(u'\u200d', u''))

##### Eliminar cualquier dígito

In [11]:
digits=re.compile(r"\d")
df["tweet"]=df["tweet"].apply(lambda x: digits.sub("",x))

##### Eliminar signos de puntuación

In [12]:
signos=re.compile(r'[!‼¡ª¿→↘↪…—"“”°‘’\$%&\'()*+,\-.\/:;=#@?\[\\\]^_`{|}~]+')
df["tweet"]=df["tweet"].apply(lambda x: signos.sub(" ",x))

##### Eliminar emojis

In [13]:
emojis= re.compile("["
                   u"\U0001F600-\U0001F9F9"  # emoticons
                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                   u"\U00002702-\U000027B0"
                   u"\U000024C2-\U0001F251"
                   u"\U000020E3-\U000029E9"
                   "]+", flags=re.UNICODE)
df["tweet"]=df["tweet"].apply(lambda x: emojis.sub(" ",x))

##### Eliminar risas

In [14]:
risa=re.compile(r'[jhea]{4,}')
df["tweet"]=df["tweet"].apply(lambda x: risa.sub(" ",x))

##### Eliminar letras repetidas, permitiendo que se repita la "E" la "L"  la "R" y la "C"

In [15]:
def rep(text):
    grp = text.group(0)
    if len(grp) > 1:
        return grp[0:1]
def cambio_letras(rep,texto):
    conver = re.sub(r"[r]{2,}", "rr", texto)
    conver = re.sub(r"[e]{2,}", "ee", conver)
    conver = re.sub(r"[l]{2,}", "ll", conver)
    conver = re.sub(r"[c]{2,}", "cc", conver)
    conver = re.sub(r'([^lerc])\1{1,}', rep, conver) 
    return conver

df["tweet"]=df["tweet"].apply(lambda x : cambio_letras(rep,x))

##### Eliminar espacios y sustituirlos por espacios sencillos

In [16]:
espacio=re.compile(r"[\s]+")
df["tweet"]=df["tweet"].apply(lambda x: espacio.sub(" ",x))

##### Sustituir algunas letras por su equivalente

In [17]:
x_re=re.compile(r"\s[x]\s")
df["tweet"]=df["tweet"].apply(lambda x: x_re.sub(" por ",x))
q_re=re.compile(r"\s[q]\s")
df["tweet"]=df["tweet"].apply(lambda x: q_re.sub(" que ",x))
d_re=re.compile(r"\s[d]\s")
df["tweet"]=df["tweet"].apply(lambda x: d_re.sub(" de ",x))
k_re=re.compile(r"\s[k]\s")
df["tweet"]=df["tweet"].apply(lambda x: k_re.sub(" que ",x))
ke_re=re.compile(r"\s[k][e]\s")
df["tweet"]=df["tweet"].apply(lambda x: ke_re.sub(" que ",x))

##### Eliminar espacios al inicio y al final del texto

In [18]:
df["tweet"]=df["tweet"].apply(lambda x: x.strip())

##### Ordenar por fecha, de más antiguo a más reciente

In [21]:
df=df.sort_values(["fecha"])

##### Guardar una versión pickle del dataframe final

In [31]:
df.to_pickle("final_df.pkl")